In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from matplotlib import pyplot as plt
from tensorflow.python.util import compat
from tensorflow.core.protobuf import saved_model_pb2
from google.protobuf import text_format
import pprint
import json
import os


In [ ]:
! rm -rf ./models && git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 86090, done.
remote: Counting objects: 100% (1689/1689), done.
remote: Compressing objects: 100% (706/706), done.
remote: Total 86090 (delta 1071), reused 1541 (delta 965), pack-reused 84401
Receiving objects: 100% (86090/86090), 598.84 MiB | 31.14 MiB/s, done.
Resolving deltas: 100% (61578/61578), done.


In [ ]:
!cd models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python3 -m pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research
  Preparing metadata (setup.py) ... done
  Using cached avro-python3-1.10.2.tar.gz (38 kB)
  Preparing metadata (setup.py) ... done
  Using cached apache_beam-2.48.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (14.3 MB)
  Using cached lvis-0.5.3-py3-none-any.whl (14 kB)
  Using cached tf_models_official-2.12.0-py2.py3-none-any.whl (2.6 MB)
  Using cached tensorflow_io-0.32.0-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (28.0 MB)
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached sacrebleu-2.2.0-py3-none-any.whl (116 kB)
  Using cached portalocker-2.7.0-py2.py3-none-any.whl (15 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)
  Using cached immutabledict-2.2.4-py3-none-any.whl (4.1 kB)
  Using cached PyYAML-5.4.1.tar.gz (175 kB)
  Installing build dependencies ... done
  Getting requireme

In [ ]:
# ! sed -i "s#ImageFont.truetype('arial.ttf', 24)#ImageFont.truetype('arial.ttf', 50)#g" ./models/research/object_detection/utils/visualization_utils.py
# ! protoc /content/models/research/object_detection/protos/*.proto --python_out=.
# ! cp /content/models/research/object_detection/packages/tf2/setup.py .
# ! pip install .
# ! pip install --use-feature=2020-resolver .
# ! pip install --upgrade pip

In [ ]:
from object_detection.utils import visualization_utils as vis_util
from object_detection.utils import dataset_util, label_map_util
from object_detection.protos import string_int_label_map_pb2

In [ ]:
def reconstruct(pb_path):
    if not os.path.isfile(pb_path):
        print("Error: %s not found" % pb_path)

    print("Reconstructing Tensorflow model")
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.compat.v1.GraphDef()
        with tf.io.gfile.GFile(pb_path, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    print("Success!")
    return detection_graph

In [ ]:
def image2np(image):
    (w, h) = image.size
    return np.array(image.getdata()).reshape((h, w, 3)).astype(np.uint8)

In [ ]:
def image2tensor(image):
    npim = image2np(image)
    return np.expand_dims(npim, axis=0)

In [ ]:
def detect(detection_graph, test_image_path):
    with detection_graph.as_default():
        gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.01)
        with tf.compat.v1.Session(graph=detection_graph,config=tf.compat.v1.ConfigProto(gpu_options=gpu_options)) as sess:
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')

            image = Image.open(test_image_path)
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image2tensor(image)}
            )

            npim = image2np(image)
            vis_util.visualize_boxes_and_labels_on_image_array(
                npim,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=15)
            plt.figure(figsize=(12, 8))
            plt.imshow(npim)
            plt.show()

In [ ]:
! kaggle datasets download -d kneroma/tacotrashdataset

100% 2.78G/2.79G [00:17<00:00, 266MB/s]
100% 2.79G/2.79G [00:17<00:00, 172MB/s]


In [ ]:
!unzip -qq tacotrashdataset.zip

In [ ]:
path = '/content/data'
annotation_file = os.path.join(path, 'annotations.json')

In [ ]:
with open(annotation_file) as json_file:
    data = json.load(json_file)

categories = data['categories']

In [ ]:
print('Building label map from examples')

labelmap = string_int_label_map_pb2.StringIntLabelMap()
for idx,category in enumerate(categories):
    item = labelmap.item.add()

    item.id = int(category['id'])+1
    item.name = category['name']

with open('./labelmap.pbtxt', 'w') as f:
    f.write(text_format.MessageToString(labelmap))

print('Label map witten to labelmap.pbtxt')

with open('./labelmap.pbtxt') as f:
    pprint.pprint(f.readlines())

Building label map from examples
Label map witten to labelmap.pbtxt
['item {\n',
 '  name: "Aluminium foil"\n',
 '  id: 1\n',
 '}\n',
 'item {\n',
 '  name: "Battery"\n',
 '  id: 2\n',
 '}\n',
 'item {\n',
 '  name: "Aluminium blister pack"\n',
 '  id: 3\n',
 '}\n',
 'item {\n',
 '  name: "Carded blister pack"\n',
 '  id: 4\n',
 '}\n',
 'item {\n',
 '  name: "Other plastic bottle"\n',
 '  id: 5\n',
 '}\n',
 'item {\n',
 '  name: "Clear plastic bottle"\n',
 '  id: 6\n',
 '}\n',
 'item {\n',
 '  name: "Glass bottle"\n',
 '  id: 7\n',
 '}\n',
 'item {\n',
 '  name: "Plastic bottle cap"\n',
 '  id: 8\n',
 '}\n',
 'item {\n',
 '  name: "Metal bottle cap"\n',
 '  id: 9\n',
 '}\n',
 'item {\n',
 '  name: "Broken glass"\n',
 '  id: 10\n',
 '}\n',
 'item {\n',
 '  name: "Food Can"\n',
 '  id: 11\n',
 '}\n',
 'item {\n',
 '  name: "Aerosol"\n',
 '  id: 12\n',
 '}\n',
 'item {\n',
 '  name: "Drink can"\n',
 '  id: 13\n',
 '}\n',
 'item {\n',
 '  name: "Toilet tube"\n',
 '  id: 14\n',
 '}\n',
 'it

In [ ]:
label_map = label_map_util.load_labelmap('labelmap.pbtxt')
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=60, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
category_index

{1: {'id': 1, 'name': 'Aluminium foil'},
 2: {'id': 2, 'name': 'Battery'},
 3: {'id': 3, 'name': 'Aluminium blister pack'},
 4: {'id': 4, 'name': 'Carded blister pack'},
 5: {'id': 5, 'name': 'Other plastic bottle'},
 6: {'id': 6, 'name': 'Clear plastic bottle'},
 7: {'id': 7, 'name': 'Glass bottle'},
 8: {'id': 8, 'name': 'Plastic bottle cap'},
 9: {'id': 9, 'name': 'Metal bottle cap'},
 10: {'id': 10, 'name': 'Broken glass'},
 11: {'id': 11, 'name': 'Food Can'},
 12: {'id': 12, 'name': 'Aerosol'},
 13: {'id': 13, 'name': 'Drink can'},
 14: {'id': 14, 'name': 'Toilet tube'},
 15: {'id': 15, 'name': 'Other carton'},
 16: {'id': 16, 'name': 'Egg carton'},
 17: {'id': 17, 'name': 'Drink carton'},
 18: {'id': 18, 'name': 'Corrugated carton'},
 19: {'id': 19, 'name': 'Meal carton'},
 20: {'id': 20, 'name': 'Pizza box'},
 21: {'id': 21, 'name': 'Paper cup'},
 22: {'id': 22, 'name': 'Disposable plastic cup'},
 23: {'id': 23, 'name': 'Foam cup'},
 24: {'id': 24, 'name': 'Glass cup'},
 25: {'i

In [ ]:
# physical_devices = tf.config.experimental.list_physical_devices("GPU")
# if physical_devices:
#     tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
! kaggle datasets download -d bouweceunen/trained-models-taco-trash-annotations-in-context

100% 111M/111M [00:00<00:00, 134MB/s] 
100% 111M/111M [00:00<00:00, 118MB/s]


In [ ]:
! unzip -qq /content/trained-models-taco-trash-annotations-in-context.zip

In [ ]:
detection_graph = reconstruct("/content/ssd_mobilenet_v2_coco_2018_03_29.pb")

Reconstructing Tensorflow model
Success!
